# Import Packages
Lets load all the needed packages for this notebook:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [ ]:
import tensorflow as tf
tf.__version__

# The Dataset
For this notebook we will use Tabular Playground Series - Mar 2021.

Let's define the path to the dataset:

In [ ]:
data = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')

In [ ]:
data.info()

# Quick Look at the Data
Let’s take a look at the top five rows:

In [ ]:
data.head()

In [ ]:
data.drop('row_id', axis = 1, inplace=True)

# Check if there is null values

In [ ]:
data.isna().sum()

In [ ]:
country = data['country'].unique()
print('Unique value of country column: ',country)

In [ ]:
# figure(figsize=(8, 6), dpi=80)
data.value_counts(data['country']).plot.bar()
plt.title('Data Distribution')
plt.xlabel('Country')
plt.ylabel('counts')

plt.show()

In [ ]:
store = data['store'].unique()
print('Unique value of store column: ', store)


data.value_counts(data['store']).plot.bar()
plt.title('Data Distribution')
plt.xlabel('Store')
plt.ylabel('counts')

plt.show()

In [ ]:
product = data['product'].unique()
print('Unique value of product column: ', product)

data.value_counts(data['product']).plot.bar()
plt.title('Data Distribution')
plt.xlabel('Product')
plt.ylabel('counts')

plt.show()

In [ ]:
data['date'] = pd.to_datetime(data['date'])

data.info()

In [ ]:
# Create new columns
data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year

data.info()

In [ ]:
data.head()

In [ ]:
data.drop(['date'], axis = 1, inplace = True)
data.head()

# Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

le_cols = ['country', 'store', 'product']

def convert2num(X_new):

    label_encoder = LabelEncoder()

    for col in le_cols:
        X_new[col] = label_encoder.fit_transform(X_new[col])
        
    return X_new

In [ ]:
data = convert2num(data)

data[le_cols].head()

In [ ]:
data.head()

In [ ]:
data['year'] = data['year']/ 2022

# Split Data

In [ ]:
data.drop(['day'], axis = 1, inplace = True)
X = data.drop('num_sold', axis=1).to_numpy()
y = data['num_sold'].to_numpy()

X.shape, y.shape

In [ ]:
X[:5]

# Splitting traning set

In [ ]:
from sklearn.model_selection import train_test_split

tf.random.set_seed(42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Building and Training our model

In [ ]:
# let's build a model to find patterns in it

# Set random seed
tf.random.set_seed(42)

# 1. Create a model
model_1 = tf.keras.Sequential([
           tf.keras.layers.Dense(500, activation='relu'),
           tf.keras.layers.Dense(250, activation='relu'),
           tf.keras.layers.Dense(100, activation='relu'),
           tf.keras.layers.Dense(10, activation='relu'), 
           tf.keras.layers.Dense(1)
])

# 2. Comile the model
model_1.compile(loss=tf.keras.losses.mae,
                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                 metrics=['MAE'])

# 3. Fit the model
history = model_1.fit(X_train, 
                      y_train, 
                      epochs=50,
                      verbose = 1,
                      validation_data=(X_test, y_test))

In [ ]:
pd.DataFrame(history.history).plot()
plt.ylabel("loss")
plt.xlabel("epochs")

In [ ]:
y_p = model_1.predict(X_test)

In [ ]:
y_p[:5], y_test[:5]

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_p)

In [ ]:
model_1.summary()

In [ ]:
# Let's check out a way of viewing our deep learning models
from tensorflow.keras.utils import plot_model

# See the inputs and outputs of each layer
plot_model(model_1, show_shapes=True)

# Test Data

In [ ]:
df_test =  pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')

In [ ]:
test_passengerIds = df_test['row_id'].values
df_test.drop('row_id', axis = 1, inplace=True)
df_test.head()

In [ ]:
df_test['date'] = pd.to_datetime(df_test['date'])

df_test.info()

In [ ]:
# Create new columns
df_test['day'] = df_test['date'].dt.day
df_test['month'] = df_test['date'].dt.month
df_test['year'] = df_test['date'].dt.year

df_test.info()

In [ ]:
df_test.drop(['date'], axis = 1, inplace = True)
df_test.head()

In [ ]:
df_test = convert2num(df_test)

df_test[le_cols].head()

In [ ]:
df_test['year'] = df_test['year']/ 2022
df_test.head()

In [ ]:
df_test.drop(['day'], axis = 1, inplace = True)
df_test = df_test.to_numpy()
df_test[:5]

In [ ]:
y_pred = model_1.predict(df_test)[:, 0]

In [ ]:
y_pred[:5]

In [ ]:
y_pred = np.array(y_pred)

In [ ]:
y_pred.shape

In [ ]:
test_passengerIds.shape

In [ ]:
output = pd.DataFrame({'row_id':test_passengerIds, 'num_sold': y_pred})
output.to_csv('submission.csv', index=False)

In [ ]:
output